# Incidentes con armas

## Análisis descriptivo

Carga de datos

In [ ]:
import pandas as pd
gun_violence_df = pd.read_csv('../data/raw/gun-violence-data_01-2013_03-2018.csv')

Pequeña muestra de los datos

In [ ]:
gun_violence_df.head()

Forma de los datos

In [ ]:
gun_violence_df.shape

Variables y sus tipos

In [ ]:
gun_violence_df.dtypes

Resumen de información de las variables

In [ ]:
gun_violence_df.info()

Descripción estadística de los datos

In [ ]:
gun_violence_df.describe()

Finalmente, hacemos uso de la herramienta Pandas Profiling para dar un resumen de nuestro dataset.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(gun_violence_df, title='Gun Violence Incidents', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/gun_violence_incidents_precleaning.html')

## Limpieza

### Tratamiento de los valores perdidos

Primero, veremos en qué columnas existen valores nulos y cuántos de ellos hay.

In [ ]:
gun_violence_df.isnull().sum()

Por tanto, las columnas con valores nulos son las siguientes:

In [ ]:
null_columns = gun_violence_df.columns[gun_violence_df.isnull().any()]
null_columns

Y sus tipos de datos son:

In [ ]:
gun_violence_df[null_columns].dtypes

Ahora, generaremos la proporción de valores nulos en cada columna para una visión a más alto nivel.

In [ ]:
null_proportions_df = pd.DataFrame(gun_violence_df[null_columns].isnull().sum() / gun_violence_df.shape[0], columns=['proportion']).sort_values(by='proportion', ascending=False)
null_proportions_df

Una estrategia típica a seguir para la imputación de valores nulos es usar la media de la variable en caso de ser numérica, y la moda en caso de ser categórica.

Las variables `participant_relationship`, `location_description` y `participant_name` poseen más de un 50% de nulos. Por tanto, no tiene sentido la imputación de la moda (por ser categóricas), ya que esta no representa a la población, pues hay más valores nulos que completos. Por tanto, introduciremos un valor `Unknown`.

In [ ]:
# Impute value 'Unknown' for null values in columns 'participant_relationship', 'location_description' and 'participant_name'
gun_violence_df['participant_relationship'].fillna('Unknown', inplace=True)
gun_violence_df['location_description'].fillna('Unknown', inplace=True)
gun_violence_df['participant_name'].fillna('Unknown', inplace=True)

Las variables `gun_stolen`, `gun_type` y `n_guns_involved` están directamente relacionadas. Según el número de armas involucradas, el tipo y si han sido robadas o no para cada una se indican en las otras dos columnas. Por ello, cuando el número de armas involucradas es nulo, las otras dos también. Por tanto, debemos seguir la misma estrategia para las tres columnas.

Sin embargo, `n_guns_involved` es una variable numérica, mientras que `gun_stolen` y `gun_type` son categóricas. Por ello, tampoco tiene sentido utilizar o bien la media o bien la moda para todas.

Finalmente, hemos valorado dos opciones:

- Introducir un valor `Unknown`.
- Utilizar una estrategia particular para el dominio.

Hemos decidido crear una estrategia propia para nuestro problema.

`gun_stolen` y `gun_type` están formateados de la siguiente manera:

    "0::<valor>||1::<valor>||...||<nº de armas involucradas - 1>::<valor>"

Además, en algunos registros existe una errata en la que se usa ':' en lugar de '::', o '|' en lugar de '||'.

Por tanto, nuestra estrategia consiste en:

1. Extraer estos valores correspondientes a cada arma involucrada del conjunto de datos.
2. Reemplazar los valores nulos de `n_guns_involved` por su media redondeada a un entero.
3. Reemplazar los valores nulos de `gun_stolen` y `gun_type` por una cadena en el formato establecido compuesta por _n_ armas, siendo _n_ la media redondeada que se ha imputado en los valores nulos de `n_guns_involved`, y cada una de ellas con el valor más frecuente para las armas involucradas.

In [ ]:
from statistics import mode

# Function to extract individual values from the given format
def extract_values(column):
    values = []
    for row in column:
        if pd.isnull(row):
            continue
        # Remove '||' and '|' separators
        no_bars = list(filter(None, row.split('|')))
        for item in no_bars:
            if ':' in item:
                # Remove '::' and ':' separators
                no_separator = list(filter(None, item.split(':')))
                values.append(no_separator[1])
            # incident_characteristics column has a different format
            else:
                values.append(item)
        
    return values

# Extract modes for gun_stolen and gun_type columns
gun_stolen_mode = mode(extract_values(gun_violence_df['gun_stolen']))
gun_type_mode = mode(extract_values(gun_violence_df['gun_type']))

print("Moda de 'gun_stolen':", gun_stolen_mode)
print("Moda de 'gun_type':", gun_type_mode)

In [ ]:
n_guns_involved_mean = gun_violence_df['n_guns_involved'].mean()
n_guns_involved_mean = round(n_guns_involved_mean, 0)
print("Media redondeada de 'n_guns_involved':", n_guns_involved_mean)

Ahora, aplicaremos la estrategia mencionada.

In [ ]:
def format_values(values):
    formatted_values = ""
    n_values = len(values)
    for i in range(n_values):
        if i == n_values - 1:
            formatted_values += str(i) + "::" + str(values[i])
        else:
            formatted_values += str(i) + "::" + str(values[i]) + "||"
    return formatted_values

# Impute values for null values in columns 'gun_stolen', 'gun_type' and 'n_guns_involved'
gun_violence_df['n_guns_involved'].fillna(n_guns_involved_mean, inplace=True)
gun_violence_df['gun_stolen'].fillna(format_values([gun_stolen_mode]), inplace=True)
gun_violence_df['gun_type'].fillna(format_values([gun_type_mode]), inplace=True)

Observando el resto de columnas con valores nulos, vemos que hay varias relacionadas con los participantes de los incidentes. Todas estas siguen el mismo formato que se expuso anteriormente. Por ello, hemos decidido volver a hacer uso de la extracción de valores de estas cadenas. Posteriormente, insertaremos la moda o la media redondeada (si son valores categóricos o numéricos, respectivamente) con el mismo formato.

In [ ]:
# Calculate mode for all the participant-related columns except 'participant_age'
participant_age_group_mode = mode(extract_values(gun_violence_df['participant_age_group']))
participant_gender_mode = mode(extract_values(gun_violence_df['participant_gender']))
participant_type_mode = mode(extract_values(gun_violence_df['participant_type']))
participant_status_mode = mode(extract_values(gun_violence_df['participant_status']))
print("Moda de 'participant_age_group':", participant_age_group_mode)
print("Moda de 'participant_gender':", participant_gender_mode)
print("Moda de 'participant_type':", participant_type_mode)
print("Moda de 'participant_status':", participant_status_mode)

from statistics import mean
# Calculate mean for 'participant_age'
participant_ages = extract_values(gun_violence_df['participant_age'])
participant_ages = [int(age) for age in participant_ages]
participant_age_mean = mean(participant_ages)
participant_age_mean = round(participant_age_mean, 0)
print("Media redondeada de 'participant_age':", participant_age_mean)

In [ ]:
# Impute values for null values in participant-related columns
gun_violence_df['participant_age_group'].fillna(format_values([participant_age_group_mode]), inplace=True)
gun_violence_df['participant_gender'].fillna(format_values([participant_gender_mode]), inplace=True)
gun_violence_df['participant_type'].fillna(format_values([participant_type_mode]), inplace=True)
gun_violence_df['participant_status'].fillna(format_values([participant_status_mode]), inplace=True)
gun_violence_df['participant_age'].fillna(format_values([int(participant_age_mean)]), inplace=True)

La columna `notes` consiste de anotaciones con información adicional del incidente en formato de texto. Por ello, la mayoría de sus valores son únicos.

In [ ]:
# Show amount and proportion of unique values for 'notes' column
notes_unique_values = gun_violence_df['notes'].unique()
print("Cantidad de valores únicos en 'notes':", len(notes_unique_values))
print("Proporción de valores únicos en 'notes':", len(notes_unique_values) / gun_violence_df.shape[0])

Además, la información más relevante de estas notas (número de heridos, de muertos...) ya está incluida en otras variables del registro. Por ello, se ha decidido introducir un valor `No Notes` con el fin de no introducir ruido en la variable mediante otros métodos.

In [ ]:
gun_violence_df['notes'].fillna('No Notes', inplace=True)

Las columnas `congressional_district`, `state_house_district`, `state_senate_district` son códigos de distrito dentro de cada estado. Por tanto, aunque dos registros tengan el mismo código de distrito para el congreso (por ejemplo), si están en distintos estados tienen significados totalmente distintos.

Por esta razón, hemos decidido reemplazar los valores nulos de estas columnas por la moda de cada una de ellas dentro del estado correspondiente al registro.

In [ ]:
# Fill null values in `congressional_district`, `state_house_district`, `state_senate_district` with the mode for the corresponding state
states = gun_violence_df['state'].unique()
state_congressional_district_mode = {}
state_house_district_mode = {}
state_senate_district_mode = {}
for state in states:
    state_congressional_district_mode[state] = mode(gun_violence_df[gun_violence_df['state'] == state]['congressional_district'])
    state_house_district_mode[state] = mode(gun_violence_df[gun_violence_df['state'] == state]['state_house_district'])
    state_senate_district_mode[state] = mode(gun_violence_df[gun_violence_df['state'] == state]['state_senate_district'])

# Print modes for Alabama as an example
print("Moda de 'congressional_district' para 'Alabama':", state_congressional_district_mode['Alabama'])
print("Moda de 'state_house_district' para 'Alabama':", state_house_district_mode['Alabama'])
print("Moda de 'state_senate_district' para 'Alabama':", state_senate_district_mode['Alabama'])


In [ ]:
gun_violence_df['congressional_district'].fillna(gun_violence_df['state'].map(state_congressional_district_mode), inplace=True)
gun_violence_df['state_house_district'].fillna(gun_violence_df['state'].map(state_house_district_mode), inplace=True)
gun_violence_df['state_senate_district'].fillna(gun_violence_df['state'].map(state_senate_district_mode), inplace=True)

Aun después de esto, hay algunos estados que cuyas columnas `state_house_district` y `state_senate_district` son todas nulas, por lo que siguen teniendo valores nulos.

Para estas variables, introduciremos un valor `-1` que indicará que es desconocido.

In [ ]:
gun_violence_df['state_house_district'].fillna(-1, inplace=True)
gun_violence_df['state_senate_district'].fillna(-1, inplace=True)

Las columnas `longitude` y `latitude` son las coordenadas del accidente. Para solventar los valores faltantes, utilizaremos la media para el estado que corresponda.

In [ ]:
# Fill null values in `latitude` and `longitude` with the mean for the corresponding state
state_latitude_mean = {}
state_longitude_mean = {}
for state in states:
    state_latitude_mean[state] = gun_violence_df[gun_violence_df['state'] == state]['latitude'].mean()
    state_longitude_mean[state] = gun_violence_df[gun_violence_df['state'] == state]['longitude'].mean()

# Print means for Alabama as an example
print("Media de 'latitude' para 'Alabama':", state_latitude_mean['Alabama'])
print("Media de 'longitude' para 'Alabama':", state_longitude_mean['Alabama'])

In [ ]:
gun_violence_df['latitude'].fillna(gun_violence_df['state'].map(state_latitude_mean), inplace=True)
gun_violence_df['longitude'].fillna(gun_violence_df['state'].map(state_longitude_mean), inplace=True)

La variable `address` es una variable referente a la ubicación demasiado concreta, por lo que la gran mayoría de sus valores son únicos.

In [ ]:
address_unique_values = gun_violence_df['address'].unique()
print("Cantidad de valores únicos en 'address':", len(address_unique_values))
print("Proporción de valores únicos en 'address':", len(address_unique_values) / gun_violence_df.shape[0])

Por tanto, ya que ni la media ni la moda tienen sentido en este caso, hemos escogido utilizar un valor `Unknown` de nuevo.

In [ ]:
gun_violence_df['address'].fillna('Unknown', inplace=True)

In [ ]:
source_url_unique_values = gun_violence_df['source_url'].unique()
print("Cantidad de valores únicos en 'source_url':", len(source_url_unique_values))
print("Proporción de valores únicos en 'source_url':", len(source_url_unique_values) / gun_violence_df.shape[0])

sources_unique_values = gun_violence_df['sources'].unique()
print("Cantidad de valores únicos en 'sources':", len(sources_unique_values))
print("Proporción de valores únicos en 'sources':", len(sources_unique_values) / gun_violence_df.shape[0])

Por tanto, aplicaremos la misma técnica: la introducción de un valor `Unknown`.

Para las variables `url_source` y `sources` tenemos también una gran cantidad de valors únicos por su significado.

In [ ]:
gun_violence_df['source_url'].fillna('Unknown', inplace=True)
gun_violence_df['sources'].fillna('Unknown', inplace=True)

Por último, la variable `incident_characteristics` presenta el mismo formato previamente mencionado, común a muchas otras. Por ello, sustituiremos los valores nulos con la moda, respetando de nuevo el formato establecido.

In [ ]:
incident_characteristics_mode = mode(extract_values(gun_violence_df['incident_characteristics']))
print("Moda de 'incident_characteristics':", incident_characteristics_mode)

In [ ]:
gun_violence_df['incident_characteristics'].fillna(format_values([incident_characteristics_mode]), inplace=True)

Finalmente, conseguimos no tener valores nulos en el conjunto de datos al completo.

In [ ]:
gun_violence_df.isnull().sum()

### Tratamiento de outliers

A continuación, trataremos los valores extremos o _outliers_ en las columnas no categóricas.

In [ ]:
numerical_columns = gun_violence_df.select_dtypes(include=['int64', 'float64']).columns
numerical_columns

De las columnas de tipo numérico, `incident_id` es un identificador único. Las variables `congressional_district`, `state_house_district` y `state_senate_district`, a pesar de ser numéricas _a priori_, realmente representan una variable categórica. Por ello, no entrarán en el cálculo de valores extremos.

Además, las variables `latitude` y `longitude` son datos espaciales que, al tratar el conjunto de datos la totalidad de los Estados Unidos, nos encontramos con un contexto geográfico muy amplio. Por ello, es natural encontrar latitudes y longitudes extremas (por ejemplo, las del estado de Alaska), y modificarlas podría distorsionar la realidad.

Además, la variable `participant_age` es también una variable numérica, aunque se encuentra en el formato típico de estas variables en nuestro conjunto de datos. Por ello, extraeremos los valores individuales de cada participante.

Por tanto, se tratarán los _outliers_ de las variables `n_killed`, `n_injured`, `n_guns_involved` y `participant_age`.

In [ ]:
# Extract participant ages
participant_ages = extract_values(gun_violence_df['participant_age'])
participant_ages = [int(age) for age in participant_ages]

Primero, visualizaremos los _outliers_.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import math
 
# Identify columns to plot
numerical_columns = ['n_killed', 'n_injured', 'n_guns_involved']
numerical_data = {
    'n_killed': gun_violence_df['n_killed'],
    'n_injured': gun_violence_df['n_injured'],
    'n_guns_involved': gun_violence_df['n_guns_involved'],
    'participant_age': participant_ages
}

# Plot boxplot for numerical_data
fig, axes = plt.subplots(math.ceil(len(numerical_data) / 2), 2, figsize=(15, 15))
axes = axes.flatten()

for ax, (col, data) in zip(axes, numerical_data.items()):
    sns.boxplot(data, ax=ax)
    ax.set_title(col)

fig.tight_layout()

Los _boxplots_ nos muestran los datos que están fuera del 50% medio como puntos. Gracias a esta visualización, podemos ver que existen valores claramente erróneos, como edades de más de 100 años.

Ahora, vamos a calcular los _outliers_ como los valores que se alejan 3 o más desviaciones estándar de la media (usando el método estadístico de _z-score_).

In [ ]:
import numpy as np 

outliers = {
    'n_killed': [],
    'n_injured': [],
    'n_guns_involved': [],
    'participant_age': []
}
# Calculate outliers for numerical_data as values that are 3 standard deviations away from the mean
for col, data in numerical_data.items():
    data = np.array(data)
    mean = np.mean(data)
    std = np.std(data)
    
    threshold = 3
    col_outliers = []
    for x in data:
        z_score = (x - mean) / std
        if abs(z_score) > threshold:
            col_outliers.append(x)
    
    outliers[col] = col_outliers
    
    print("Columna: ", col)
    print("Media: ", mean)
    print("Desviación estándar: ", std)
    print("Número de outliers: ", len(col_outliers))
    print("Proporción respecto al número de filas: ", len(col_outliers) / gun_violence_df.shape[0])
    print("Outlier mínimo: ", min(col_outliers))
    print("Outlier máximo: ", max(col_outliers), "\n")

In [ ]:
# Remove outliers from gun_violence_df
for col in numerical_columns:
    gun_violence_df = gun_violence_df[~gun_violence_df[col].isin(outliers[col])]

outliers["participant_age"] = [str(age) for age in outliers["participant_age"]]
substrings_to_remove = "|".join(outliers["participant_age"])
gun_violence_df = gun_violence_df[~gun_violence_df['participant_age'].str.contains(substrings_to_remove)]

In [ ]:
participant_ages = extract_values(gun_violence_df['participant_age'])
participant_ages = [int(age) for age in participant_ages]

numerical_data = {
    'n_killed': gun_violence_df['n_killed'],
    'n_injured': gun_violence_df['n_injured'],
    'n_guns_involved': gun_violence_df['n_guns_involved'],
    'participant_age': participant_ages
}

# Plot boxplot for numerical_data without outliers
fig, axes = plt.subplots(math.ceil(len(numerical_data) / 2), 2, figsize=(15, 15))
axes = axes.flatten()

for ax, (col, data) in zip(axes, numerical_data.items()):
    data = np.array(data)
    sns.boxplot(data, ax=ax)
    ax.set_title(col)

Podemos ver que en los nuevos _boxplots_ siguen existiendo _outliers_. Esto es porque utiliza el método del IQR, mientras que nosotros hemos utilizado el método del _z-score_, por lo que son distintos. Sin embargo, esto nos muestra que hemos reducido considerablemente el número de valores extremos.

### Profiling post-limpieza

De nuevo, volvemos hacer uso del profiling para ver un resumen de nuestros datos tras la limpieza.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(gun_violence_df, title='Gun Violence Incidents', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/gun_violence_incidents_postcleaning.html')

## Integración

Ahora, procederemos a resolver los problemas de representación y codificación que se presenten en el conjunto de datos, de cara a seguir un formato común con las demás fuentes.

Las variables que pueden presentarnos problemas de formato son:

- La fecha, por ser esta una variable en la que puede haber diferencias de formato dependiendo de la fuente. Además, dependiendo de la hipótesis tendremos una granularidad de día o de año.
- `gun_type`, `gun_stolen` y las variables referentes a los participantes (`participant_age`, `participant_gender`...), por tener un formato propio al conjunto de datos.

Para las segundas, hemos decidido representarlas como una lista, ya que Pandas lo permite.

In [ ]:
# Function to extract individual values from the given format, and return them as a new column with lists
def extract_values_to_lists(column):
    new_column = {column.name: []}
    for row in column:
        new_row = []
        if pd.isnull(row):
            continue
        # Remove '||' and '|' separators
        no_bars = list(filter(None, row.split('|')))
        for item in no_bars:
            if ':' in item:
                # Remove '::' and ':' separators
                no_separator = list(filter(None, item.split(':')))
                if len(no_separator) == 1:
                    print("Empty value found in column", column.name)
                    new_row.append('Unknown')
                    continue
                # participant_age column has integer values
                if column.name == 'participant_age':
                    new_row.append(int(no_separator[1]))
                else:
                    new_row.append(no_separator[1])
        new_column[column.name].append(new_row)
        
    return pd.DataFrame.from_dict(new_column)

columns_to_format = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']

for col in columns_to_format:
    gun_violence_df[col] = extract_values_to_lists(gun_violence_df[col])[col].values

In [ ]:
gun_violence_df['date'].head()

La fecha sigue el formato ISO 8601 extendido. Sin embargo, para nuestras hipótesis utilizaremos una granularidad de año. Por ello, separaremos la fecha en tres variables `year`, `month` y `day`, con el fin de poder unir nuestros datos después.

In [ ]:
# Separate date into year, month and day columns
gun_violence_df['year'] = pd.DatetimeIndex(gun_violence_df['date']).year
gun_violence_df['month'] = pd.DatetimeIndex(gun_violence_df['date']).month
gun_violence_df['day'] = pd.DatetimeIndex(gun_violence_df['date']).day

# Drop date column and reorder columns
gun_violence_df.drop(columns=['date'], inplace=True)
cols = gun_violence_df.columns.tolist()
cols = [cols[0]] + cols[-3:] + cols[1:-3]
gun_violence_df = gun_violence_df[cols]

In [ ]:
# Save gun_violence_df to csv
import os
os.makedirs('../data/interim', exist_ok=True)
gun_violence_df.to_csv('../data/interim/gun_violence_incidents.csv', index=False)

## Reducción

Nuestras hipótesis sólo se refieren al número de incidentes, y la relación con las otras fuentes de datos será mediante la fecha y el estado. Por tanto, nos quedaremos con las siguientes columnas:

In [ ]:
gun_violence_selected_columns = ['date', 'state']

Ya que los datos de 2018 sólo llegan hasta marzo, mientras que los de los demás años están completos, descartaremos también los datos de este año.

In [ ]:
gun_violence_df = gun_violence_df[gun_violence_df['year'] != 2018]

Además, dependiendo de la hipótesis para la que se usen estos datos reduciremos o no el número de registros. Para las dos primeras hipótesis, en las que usaremos solo este conjunto de datos, se quedará igual. Sin embargo, cuando se usen en conjunto con otros datos, descartaremos los datos previos al 2015, pues es el primer año para el que tenemos datos de pobreza (y poblaciones). A su vez, cuando se usen en conjunto con los datos de leyes, descartaremos los datos del estado "District of Columbia", pues no tenemos datos de leyes para este.

Esto se hará justo antes de crear las tarjetas de datos.

# Clima

## Análisis descriptivo

Los datos de clima se proveen en un formato de texto difícil de tratar, por lo que primero deberemos analizar estos ficheros y convertirlos a un DataFrame. Esto comprende varias tareas de preprocesado más complejas que con los demás conjuntos de datos, por lo que se ha decidido realizar estas tareas en un _script_ aparte.

In [ ]:
import sys
sys.path.insert(0, '../src/data')
from parse_climate_data import parse_climate_data
climate_data = ['../data/raw/temperature_data.txt', '../data/raw/precipitation_data.txt']
climate_df = parse_climate_data('../data/raw/climate_state_codes.txt', climate_data)

Pequeña muestra de los datos

In [ ]:
climate_df.head()

Forma de los datos

In [ ]:
climate_df.shape

Variables y sus tipos

In [ ]:
climate_df.dtypes

Resumen de información de las variables

In [ ]:
climate_df.info()

Descripción estadística de los datos

In [ ]:
climate_df.describe()

Finalmente, hacemos uso de la herramienta Pandas Profiling para dar un resumen de nuestro dataset.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(climate_df, title='Climate Data', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/climate_data_precleaning.html')

## Limpieza

### Tratamiento de los valores perdidos

Primero, veremos en qué columnas existen valores nulos y cuántos de ellos hay.

In [ ]:
climate_df.isnull().sum()

Por tanto, las columnas con valores nulos son las siguientes:

In [ ]:
null_columns = climate_df.columns[climate_df.isnull().any()]
null_columns

Al igual que en el anterior conjunto de datos, imputaremos la media para los valores nulos, por ser una variable numérica. Además, al ser datos climáticos que dependen del lugar (estado), utilizaremos la media del estado correspondiente al registro.

In [ ]:
# Fill missing values with the mean value of the column for the corresponding state
for col in null_columns:
    climate_df[col] = climate_df.groupby('state')[col].transform(lambda x: x.fillna(x.mean()))

Finalmente, conseguimos no tener valores nulos en el conjunto de datos al completo.

In [ ]:
climate_df.isnull().sum()

### Profiling post-limpieza

De nuevo, volvemos hacer uso del profiling para ver un resumen de nuestros datos tras la limpieza.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(climate_df, title='Climate Data', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/climate_data_postcleaning.html')

### Tratamiento de outliers

A continuación, trataremos los valores extremos o _outliers_ en las columnas no categóricas.

In [ ]:
numerical_columns = climate_df.select_dtypes(include=['int64', 'float64']).columns
numerical_columns

Ya que `year` y `month` nos indican el tiempo en el que se realizó la medición, solo trataremos los _outliers_ para `average_temperature` y `average_precipitation`.

Primero, visualizaremos los _outliers_.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import math
 
numerical_columns = numerical_columns.drop(['year', 'month'])

fig, axes = plt.subplots(math.ceil(len(numerical_columns) / 2), 2, figsize=(15, 15))
axes = axes.flatten()
for ax, col in zip(axes, numerical_columns):
    sns.boxplot(climate_df[col], ax=ax)
    ax.set_title(col)

Los _boxplots_ nos muestran los datos que están fuera del 50% medio como puntos. Gracias a esta visualización, podemos ver que existen outliers correspondientes a temperaturas bajo cero y precipitaciones muy altas.

Ahora, vamos a calcular los _outliers_ como los valores que se alejan 3 o más desviaciones estándar de la media (usando el método estadístico de _z-score_).

In [ ]:
# Calculate outliers for numerical_columns as values that are 3 standard deviations away from the mean
outliers = []
for col in numerical_columns:
    mean = climate_df[col].mean()
    std = climate_df[col].std()
    outliers = climate_df[(climate_df[col] < mean - 3*std) | (climate_df[col] > mean + 3*std)][col].values

    print("Columna: ", col)
    print("Media: ", mean)
    print("Desviación estándar: ", std)
    print("Número de outliers: ", len(outliers))
    print("Proporción respecto al número de filas: ", len(outliers) / climate_df.shape[0])
    if len(outliers) > 0:
        print("Outlier mínimo: ", min(outliers))
        print("Outlier máximo: ", max(outliers), "\n")
    else:
        print("\n")

Ya que estos valores pueden ser interesantes de analizar para ver qué ocurre en estados con temperaturas extremadamente bajas o precipitaciones extremadamente bajas, además de presentar tan pocos _outliers_, no eliminaremos ni sustituiremos estos valores. Además, cuando los pasemos a una unidad estándar serán menos aún.

## Integración

Los datos de temperatura de este conjunto se muestran en grados Fahrenheit. Nosotros estamos más acostumbrados a grados Celsius, por lo que los transformaremos a estos.

In [ ]:
climate_df['average_temperature'] = (climate_df['average_temperature'] - 32) * 5/9

Los datos de precipitaciones se presentan en centésimas de pulgada, que pasaremos a mm (unidad estándar).

In [ ]:
climate_df['average_precipitation'] = climate_df['average_precipitation'] * 0.254

In [ ]:
# Save climate_df to csv
climate_df.to_csv('../data/interim/climate_data.csv', index=False)

## Reducción

Ya que nuestros datos de incidentes van desde 2014 a 2017, descartaremos los datos de otros años.

In [ ]:
# Remove data previous to 2014 and after 2017 from climate_df
climate_df = climate_df[climate_df['year'] > 2013]
climate_df = climate_df[climate_df['year'] < 2018]

# Pobreza

## Análisis descriptivo

Carga de datos

In [ ]:
import pandas as pd
poverty_df = pd.read_csv('../data/raw/poverty_data.csv')

Pequeña muestra de los datos

In [ ]:
poverty_df.head()

Forma de los datos

In [ ]:
poverty_df.shape

Variables y sus tipos

In [ ]:
poverty_df.dtypes

Resumen de información de las variables

In [ ]:
poverty_df.info()

Descripción estadística de los datos

In [ ]:
poverty_df.describe()

Finalmente, hacemos uso de la herramienta Pandas Profiling para dar un resumen de nuestro dataset.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(poverty_df, title='Poverty Data', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/poverty_data_precleaning.html')

## Limpieza

### Tratamiento de los valores perdidos

Primero, veremos en qué columnas existen valores nulos y cuántos de ellos hay.

In [ ]:
poverty_df.isnull().sum()

Por tanto, las columnas con valores nulos son las siguientes:

In [ ]:
null_columns = poverty_df.columns[poverty_df.isnull().any()]
null_columns

Al igual que en el anterior conjunto de datos, imputaremos la media para los valores nulos, por ser una variable numérica. Además, al ser datos demográficos que dependen del estado, utilizaremos la media del estado correspondiente al registro.

In [ ]:
# Fill missing values with the mean value of the column for the corresponding state
for col in null_columns:
    poverty_df[col] = poverty_df.groupby('state')[col].transform(lambda x: x.fillna(x.mean()))

Finalmente, conseguimos no tener valores nulos en el conjunto de datos al completo.

In [ ]:
poverty_df.isnull().sum()

### Tratamiento de outliers

A continuación, trataremos los valores extremos o _outliers_ en las columnas no categóricas.

In [ ]:
numerical_columns = poverty_df.select_dtypes(include=['int64', 'float64']).columns
numerical_columns

Descartaremos la variable `year` y trataremos los _outliers_ de las demás.

Primero, visualizaremos los _outliers_.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import math
 
numerical_columns = numerical_columns.drop('year')
# Plot boxplots for numerical columns
fig, ax = plt.subplots(math.ceil(len(numerical_columns)/3), 3, figsize=(15, 15))
for i, col in enumerate(numerical_columns):
    sns.boxplot(y=col, data=poverty_df, ax=ax[math.floor(i/3), i%3])
plt.tight_layout()

Los _boxplots_ nos muestran los datos que están fuera del 50% medio como puntos. Gracias a esta visualización, podemos ver que existen más outliers en las variables `population` y `in_poverty`. Esto es debido a que estas columnas son cantidades de habitantes, mientras que las otras son porcentajes. Esta diferencia de escalado hace que los valores extremos sean aún más extremos en estas.

Ahora, vamos a calcular los _outliers_ como los valores que se alejan 3 o más desviaciones estándar de la media (usando el método estadístico de _z-score_).

In [ ]:
# Calculate outliers for numerical_data as values that are 3 standard deviations away from the mean
outliers = {}
for col in numerical_columns:
    mean = poverty_df[col].mean()
    std = poverty_df[col].std()
    outliers[col] = poverty_df[(poverty_df[col] < mean - 3*std) | (poverty_df[col] > mean + 3*std)][col].values

    print("Columna: ", col)
    print("Media: ", mean)
    print("Desviación estándar: ", std)
    print("Número de outliers: ", len(outliers[col]))
    print("Proporción respecto al número de filas: ", len(outliers[col]) / poverty_df.shape[0])
    if len(outliers[col]) > 0:
        print("Outlier mínimo: ", min(outliers[col]))
        print("Outlier máximo: ", max(outliers[col]), "\n")
    else:
        print("\n")

Ya que los valores porcentuales no presentan _outliers_, y la renta (mediana) sólo presenta un _outlier_, no eliminaremos ningún _outlier_.

### Profiling post-limpieza

De nuevo, volvemos hacer uso del profiling para ver un resumen de nuestros datos tras la limpieza.

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(poverty_df, title='Poverty Data', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
import os
os.makedirs('../reports/descriptive_analysis', exist_ok=True)
profile.to_file(output_file='../reports/descriptive_analysis/poverty_data_postcleaning.html')

## Integración

Los datos de este conjunto ya poseen un formato adecuado, por lo que no tenemos problemas de codificación/representación que resolver.

In [ ]:
# Save poverty_df to csv
os.makedirs('../data/interim', exist_ok=True)
poverty_df.to_csv('../data/interim/poverty_data.csv', index=False)

## Reducción

A primera vista, todas las variables parecen indicadores de pobreza importantes. Por ello, generaremos la matriz de correlación y veremos si es así.

In [ ]:
# Compute correlation matrix for poverty numerical data
import numpy as np
corr = poverty_df[numerical_columns].corr('pearson')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Plot correlation matrix
plt.figure(figsize=(10, 10))
sns.heatmap(corr, annot=True, mask=mask, fmt='.2f', cmap='coolwarm')

Vemos que existe una alta correlación entre algunos pares de variables.

In [ ]:
# For each variable, show highly correlated variables (correlation > 0.5 or < -0.5)
for col in numerical_columns:
    print(col)
    print(corr[((corr[col] > 0.5) | (corr[col] < -0.5)) & (corr[col] < 1)][col], "\n")

Podemos observar que las variables están muy altamente correlacionadas. `population` e `in_poverty` están fuertemente correlacionadas, así como `poverty_rate` con `median_household_income`, `deep_poverty_rate`, `unemployment_rate` y `supplemental_poverty_measure`. Además, este `poverty_rate` tiene un coeficiente de Pearson muy cercano al umbral que hemos puesto (+-0,5) con `median_rent` y `withouth_health_insurance`.

Por tanto, nos quedaremos solo con `poverty_rate` como indicador de pobreza.

In [ ]:
poverty_selected_columns = ['year', 'state', 'poverty_rate']
poverty_selected_columns

# Leyes sobre armas

## Análisis descriptivo

Carga de datos

In [ ]:
import pandas as pd
firearm_laws_database_df = pd.read_excel('../data/raw/firearm_laws_database.xlsx')
firearm_laws_codebook_df = pd.read_excel('../data/raw/firearm_laws_codebook.xlsx')

Pequeña muestra de los datos

In [ ]:
firearm_laws_database_df.head()

In [ ]:
firearm_laws_codebook_df.head()

Forma de los datos

In [ ]:
firearm_laws_database_df.shape

In [ ]:
firearm_laws_codebook_df.shape

Variables y sus tipos

In [ ]:
firearm_laws_database_df.dtypes

In [ ]:
firearm_laws_codebook_df.dtypes

Resumen de información de las variables

In [ ]:
firearm_laws_database_df.info()

In [ ]:
firearm_laws_codebook_df.info()

Como podemos ver, este dataset es algo peculiar. Cada registro se refiere a un estado, un año, y el resto de columnas a una ley en particular, que puede tomar como valor 0 (no activa) o 1 (activa). Además, el _codebook_ que usaremos para dividir las leyes en grupos tiene todas las variables en formato de texto, exceptuando el código de la categoría.

Por estas características, creemos que no nos es útil realizar una descripción estadística de los datos.

## Limpieza

### Tratamiento de valores perdidos

Solo existen valores nulos en el _codebook_.

In [ ]:
firearm_laws_database_df.isnull().sum()

In [ ]:
firearm_laws_codebook_df.isnull().sum()

Por ser estos datos de tipo textual, los reemplazaremos con un valor `Unknown`.

In [ ]:
null_columns = firearm_laws_codebook_df.columns[firearm_laws_codebook_df.isnull().any()]
for col in null_columns:
    firearm_laws_codebook_df[col].fillna('Unknown', inplace=True)

In [ ]:
firearm_laws_codebook_df.isnull().sum()

Por las características peculiares del conjunto de datos comentadas anteriormente, creemos que no tiene sentido buscar _outliers_.

## Integración

Además, los datos que necesitaremos están correctamente formateados. Sin embargo, los nombres de las columnas en el _codebook_ están escritos de manera distinta a los demás conjuntos de datos. Para homogeneizar las fuentes, las formatearemos correctamente.

In [ ]:
firearm_laws_codebook_df.columns = firearm_laws_codebook_df.columns.str.lower().str.replace(' ', '_')

También guardaremos los ficheros en formato csv en lugar de xlsx.

In [ ]:
# Save firearm_laws_codebook_df to csv
os.makedirs('../data/interim', exist_ok=True)
firearm_laws_codebook_df.to_csv('../data/interim/firearm_laws_codebook.csv', index=False)

# Save firearm_laws_database_df to csv
os.makedirs('../data/interim', exist_ok=True)
firearm_laws_database_df.to_csv('../data/interim/firearm_laws_database.csv', index=False)

## Reducción

Para el conjunto de datos en sí (_database_), utilizaremos todas las columnas, ya que agruparemos después por tipo de ley (categoría en el _codebook_) y nos serán necesarias. Sin embargo, nos desharemos de todos los datos previos a 2014, ya que no tenemos datos de incidentes antes de ese año.

Para el _codebook_, por tanto, tomaremos sólo el código de la categoría y el nombre de la variable, para su uso después en la transformación.

In [ ]:
firearm_laws_codebook_selected_columns = ['category_code', 'variable_name']

In [ ]:
firearm_laws_database_df = firearm_laws_database_df[firearm_laws_database_df['year'] >= 2013]

# Poblaciones

## Análisis descriptivo

Carga de datos

In [ ]:
import pandas as pd
population_df = pd.read_csv('../data/raw/population_data.csv')

Pequeña muestra de los datos

In [ ]:
population_df.head()

Forma de los datos

In [ ]:
population_df.shape

Variables y sus tipos

In [ ]:
population_df.dtypes

Resumen de información de las variables

In [ ]:
population_df.info()

Descripción estadística de los datos

In [ ]:
population_df.describe()

## Limpieza

### Tratamiento de los valores perdidos

Primero, veremos si existen nulos.

In [ ]:
population_df.isnull().values.any()

Como no los hay, no necesitamos realizar más operaciones.

### Tratamiento de outliers

Ya que los valores del conjunto de datos son los habitantes de cada estado, habrá valores extremos correspondientes a los estados con más habitantes (e.g. California) o con menos (e.g. Alaska). Sin embargo, estos valores nos son necesarios, ya que los usaremos para escalar los datos de otros conjuntos para cada estado. Por tanto, no realizaremos ningún descarte de _outliers_.

De nuevo, volvemos hacer uso del profiling para ver un resumen de nuestros datos tras la limpieza.

## Integración

Este conjunto de datos presenta una variable por año para los distintos valores de población. Por ello, para coincidir con los demás datos, crearemos una variable `year` y modificaremos las demás columnas.

In [ ]:
cols = population_df.columns.tolist()

# Create 'year' empty column right after 'NAME'
cols.insert(5, 'year')

# Keep the first 5 columns
new_population_data = {col: [] for col in cols[:5]}
new_population_data['year'] = []
# Extract name without year from each of the other colums
cols_to_be_grouped = cols[8:]
for col in cols_to_be_grouped:
    new_col = col[:-4]
    new_col = new_col.replace('_', '')
    new_population_data[new_col] = []

# There are some columns with null values for 2010, which will be assigned None and handled later
null_columns = ['RBIRTH', 'RDEATH', 'RNATURALINC', 'RINTERNATIONALMIG', 'RDOMESTICMIG', 'RNETMIG']

# Repeat each row 10 times, one for each year
for i in range(population_df.shape[0]):
    # Add None values for null columns in 2010
    for col in null_columns:
        new_population_data[col].append(None)

    for j in range(10):
        new_population_data['year'].append(2010 + j)
        # For each of the old columns which end with the year, add the value to the new column
        for col in cols_to_be_grouped:
            year = int(col[-4:])
            if year == 2010 + j:
                new_col = col[:-4]
                new_col = new_col.replace('_', '')
                new_population_data[new_col].append(population_df[col][i])
        
        # Add the values of the first 5 columns
        for col in cols[:5]:
            new_population_data[col].append(population_df[col][i])

population_df = pd.DataFrame.from_dict(new_population_data)

Así, obtenemos el conjunto de datos con el nuevo formato de columnas.

In [ ]:
population_df.head()

Además, cambiaremos los nombres de las columnas a minúscula para seguir el mismo patrón en todos los conjuntos.

In [ ]:
population_df.columns = population_df.columns.str.lower()

Gracias a la nueva distribución, hemos observado que existen valores nulos para algunas variables en 2010. Los reemplazaremos por la media de la variable para ese estado.

In [ ]:
# Impute missing values with the mean value of the column for the corresponding state
null_columns = population_df.columns[population_df.isnull().any()]
for col in null_columns:
    population_df[col] = population_df.groupby('name')[col].transform(lambda x: x.fillna(x.mean()))

In [ ]:
population_df.isnull().sum()

Por tanto, nuestros datos limpios y con formato adecuado son los mismos que en crudo.

In [ ]:
os.makedirs('../data/interim', exist_ok=True)
population_df.to_csv('../data/interim/population_data.csv', index=False)

## Reducción

Para el uso que le daremos a este conjunto de datos, nos interesan solo las poblaciones de los estados (en este conjunto se incluye también Puerto Rico) desde 2014 a 2017. Por tanto, seleccionaremos estos datos.

In [ ]:
regions_to_remove = ['United States', 'Northeast Region', 'Midwest Region', 'South Region', 'West Region', 'Puerto Rico']
selected_years = [2014, 2015, 2016, 2017]
selected_population_columns = ['name', 'year', 'popestimate']

population_df = population_df[~population_df['name'].isin(regions_to_remove)]
population_df = population_df[population_df['year'].isin(selected_years)]
population_df = population_df[selected_population_columns]
population_df.head()

# Transformación

Ahora, crearemos una variable que nos indique si la fecha es fin de semana. Además, agregaremos el número de incidentes en cada estado y fecha.

In [ ]:
# Aggregation of number of incidents by state, year, month and day
table_1 = gun_violence_df.groupby(['state', 'year', 'month', 'day']).size().reset_index(name='n_incidents')

# Create column is_weekend
import datetime
table_1['is_weekend'] = table_1.apply(lambda row: 1 if datetime.datetime(row['year'], row['month'], row['day']).weekday() >= 5 else 0, axis=1)

table_1.head()

Ahora, usaremos estas nuevas variables para agrupar los datos por año y estado, creando nuevas variables que nos indiquen cuántos incidentes diarios se han dado en un cierto año y estado en fines de semana.

In [ ]:
# For each state and year, calculate the number of incidents in weekends
table_1 = table_1.groupby(['state', 'year', 'is_weekend']).agg({'n_incidents': ['sum', 'count']}).reset_index()
table_1.columns = ['state', 'year', 'is_weekend', 'n_incidents', 'n_days']

# Divide sum by count to get the incidents per day
table_1['n_incidents_per_day'] = table_1['n_incidents'] / table_1['n_days']
table_1.drop(columns=['n_incidents', 'n_days'], inplace=True)

table_1.head()

Por último, escalaremos estos incidentes diarios por 100.000 habitantes.

In [ ]:
# Get popestimate from population_df (which is the population) for each state and year and divide the number of incidents per day by 100000 to get the incidents per 100000 people

table_1 = table_1.merge(population_df, left_on=['state', 'year'], right_on=['name', 'year'])
table_1.drop(columns=['name'], inplace=True)
table_1['n_incidents_per_day'] = table_1['n_incidents_per_day'] / table_1['popestimate'] * 100000
table_1.drop(columns=['popestimate'], inplace=True)

table_1.head()

Así, generamos la tarjeta de datos para las primera hipótesis.

In [ ]:
# Save table_1 to csv
os.makedirs('../data/processed', exist_ok=True)
table_1.to_csv('../data/processed/incidents_weekend.csv', index=False)

Ahora, generaremos la segunda tarjeta de datos. Agregaremos los incidentes ocurridos en cada estado, año y mes, junto con su temperatura media. Utilizaremos también la población del estado ese año para escalar el número de incidentes.

In [ ]:
# Aggregation of number of incidents by state, year and month
table_2 = gun_violence_df.groupby(['state', 'year', 'month']).size().reset_index(name='n_incidents')

# Add average temperature for each state, year and month
table_2 = table_2.merge(climate_df, on=['state', 'year', 'month'], how='left')

# Scale the number of incidents by population (per 100,000 inhabitants) using population_df's popestimate column
table_2 = table_2.merge(population_df, left_on=['state', 'year'], right_on=['name', 'year'])
table_2.drop(columns=['name'], inplace=True)
table_2['n_incidents'] = table_2['n_incidents'] / table_2['popestimate'] * 100000
table_2.drop(columns=['popestimate'], inplace=True)

# Drop 'District of Columbia' and 'Hawaii' because it has no data for average temperature
table_2 = table_2[~table_2['state'].isin(['District of Columbia', 'Hawaii'])]

table_2.head()

In [ ]:
# Save table_2 to csv
table_2.to_csv('../data/processed/incidents_climate.csv', index=False)

Después, generaremos la tercera tarjeta de datos. De igual manera, agregaremos los incidentes ocurridos en cada estado y año, junto con su ratio de pobreza. Utilizaremos también la población del estado ese año para escalar el número de incidentes.

In [ ]:
# Aggregation of number of incidents by state and year
table_3 = gun_violence_df.groupby(['state', 'year']).size().reset_index(name='n_incidents')

table_3 = table_3[table_3['year'] > 2014]

# Add poverty_rate column to table_3
poverty_df = poverty_df[poverty_selected_columns]
table_3 = pd.merge(table_3, poverty_df, on=['state', 'year'], how='left')

# Scale the number of incidents by population (per 100,000 inhabitants) using population_df's popestimate column
table_3 = table_3.merge(population_df, left_on=['state', 'year'], right_on=['name', 'year'])
table_3.drop(columns=['name'], inplace=True)
table_3['n_incidents'] = table_3['n_incidents'] / table_3['popestimate'] * 100000
table_3.drop(columns=['popestimate'], inplace=True)

table_3.head()

Como no tenemos datos de pobreza para el año 2014, hemos eliminado los datos de incidentes de ese año en el tablón. Así, obtenemos el tablón para la tercera hipótesis.

In [ ]:
# Save table_3 to csv
os.makedirs('../data/processed', exist_ok=True)
table_3.to_csv('../data/processed/incidents_population_poverty.csv', index=False)

Por último, generaremos la cuarta tarjeta de datos. Agregaremos de nuevo los incidentes por estado y año, y los escalaremos por población. Añadiremos el número de leyes activas, y crearemos una columna por cada categoría de ley.

In [ ]:
# Aggregation of number of incidents by state and year
table_4 = gun_violence_df.groupby(['state', 'year']).size().reset_index(name='n_incidents')

# Add popestimate column from population_df to table_4 and scale the number of incidents by population (per 100,000 inhabitants)
table_4 = table_4.merge(population_df, left_on=['state', 'year'], right_on=['name', 'year'])
table_4.drop(columns=['name'], inplace=True)
table_4['n_incidents'] = table_4['n_incidents'] / table_4['popestimate'] * 100000
table_4.drop(columns=['popestimate'], inplace=True)

# Add firearm laws columns to table_4
table_4 = pd.merge(table_4, firearm_laws_database_df, on=['state', 'year'], how='left')
table_4.head()

# Create new columns for each category_code
firearm_laws_codebook_df = firearm_laws_codebook_df[firearm_laws_codebook_selected_columns]
category_columns = []
for category_code in firearm_laws_codebook_df['category_code'].unique():
    category_columns.append("laws_" + str(category_code))

for category_column in category_columns:
    table_4[category_column] = 0

# Fill category columns with the corresponding value. For a state and year, check all the variables and add 1 to the corresponding category column if the value is 1
for index, row in table_4.iterrows():
    for category_column in category_columns:
        for variable_name in firearm_laws_codebook_df[firearm_laws_codebook_df['category_code'] == int(category_column[5:])]['variable_name']:
            if row[variable_name] == 1:
                table_4.at[index, category_column] += 1

# Drop all the variables columns
for variable_name in firearm_laws_codebook_df['variable_name']:
    table_4.drop(columns=[variable_name], inplace=True)

# Drop 'District of Columbia' data
table_4 = table_4[table_4['state'] != 'District of Columbia']

table_4.head()

Así, obtenemos la tarjeta de datos para la última hipótesis.

In [ ]:
os.makedirs('../data/processed', exist_ok=True)
table_4.to_csv('../data/processed/incidents_firearm_laws.csv', index=False)

Por último, se ha decidido generar una tarjeta de datos adicional que combine todos los datos (exceptuando los de si es fin de semana o no), para poder realizar un análisis más global. Por tanto, para resolver incompatibilidades entre distintos conjuntos de datos:

- Se utilizará granularidad de año.
- Se descartarán datos de Hawaii y District of Columbia.
- Se tomarán sólo datos de 2015, 2016 y 2017.

In [ ]:
## INCIDENTS
# Aggregation of number of incidents by state and year
table_5 = gun_violence_df.groupby(['state', 'year']).size().reset_index(name='n_incidents')

# Add popestimate column from population_df to table_5 and scale the number of incidents by population (per 100,000 inhabitants)
table_5 = table_5.merge(population_df, left_on=['state', 'year'], right_on=['name', 'year'])
table_5.drop(columns=['name'], inplace=True)
table_5['n_incidents'] = table_5['n_incidents'] / table_5['popestimate'] * 100000
table_5.drop(columns=['popestimate'], inplace=True)

## CLIMATE
# Aggregation of average temperature and average precipitation by state and year (mean)
climate_df = climate_df.groupby(['state', 'year']).mean().reset_index()
climate_df.drop(columns=['month'], inplace=True)

# Add average temperature column to table_5
table_5 = pd.merge(table_5, climate_df, on=['state', 'year'], how='left')

## POVERTY
# Add poverty_rate column to table_5
poverty_df = poverty_df[poverty_selected_columns]
table_5 = pd.merge(table_5, poverty_df, on=['state', 'year'], how='left')

## FIREARM LAWS
# Add firearm laws columns to table_5
table_5 = pd.merge(table_5, firearm_laws_database_df, on=['state', 'year'], how='left')
table_5.head()

# Create new columns for each category_code
firearm_laws_codebook_df = firearm_laws_codebook_df[firearm_laws_codebook_selected_columns]
category_columns = []
for category_code in firearm_laws_codebook_df['category_code'].unique():
    category_columns.append("laws_" + str(category_code))

for category_column in category_columns:
    table_5[category_column] = 0

# Fill category columns with the corresponding value. For a state and year, check all the variables and add 1 to the corresponding category column if the value is 1
for index, row in table_5.iterrows():
    for category_column in category_columns:
        for variable_name in firearm_laws_codebook_df[firearm_laws_codebook_df['category_code'] == int(category_column[5:])]['variable_name']:
            if row[variable_name] == 1:
                table_5.at[index, category_column] += 1

# Drop all the variables columns
for variable_name in firearm_laws_codebook_df['variable_name']:
    table_5.drop(columns=[variable_name], inplace=True)

# Drop 'District of Columbia' and 'Hawaii' data
table_5 = table_5[~table_5['state'].isin(['District of Columbia', 'Hawaii'])]

# Drop data before 2015 and after 2017
table_5 = table_5[table_5['year'] > 2014]
table_5 = table_5[table_5['year'] < 2018]

table_5.head()

In [ ]:
# Save table_5 to csv
os.makedirs('../data/processed', exist_ok=True)
table_5.to_csv('../data/processed/incidents_combined.csv', index=False)